## How GNN work

In this tutorial, we will explore graph neural networks and graph convolutions. **Graphs are a super general representation of data with intrinsic structure**.

# Decomposing features(signal) and structure

The key concept to understand graphs is **the decomposition of structure and signal (features)**, which make them so powerful and general methods.

`Graphs are not any different: they are data with decomposed structure and signal information.`

## Real-world signals that we can model with graphs

As long as you can define these two representations, you can model anything you want with graphs.

Formally, the words or pixels are simply nodes, denoted by $N$. The connectivity/structure will be defined by a $N \times N$ matrix, the so-called **Adjacency matrix** $A$. The element $i, j$ of $A$ will tell us if node $i$ is connected to node $j$.

The signal/features for eaach node will be $X \in R^{N \times F}$. $F$ is the number of features.

![example](https://theaisummer.com/static/692e414df0c37c4becd39593f5ca8d2d/82b28/graph-structure-signal.png)

**Tips**: Note that the diagonal of $A$ is shown to contain ones, which is usually the case in graph neural networks **for training stability reasons**, although in the general case it has zeros, indicating no-self connections.

## The basic maths for processing graph-structured data

A very important and practical feature is the **degree** of each node, which is simply **the number of nodes that it is connected to**.

If $A$ is binary the degree corresponds to the number of neighbors in the graph.

**Tips**: Since the degree corresponds to some kind of feature that is linked to the node, it is more convenient to place the degree vector in a diagonal $N \times N$ matrix:

In [9]:
import torch

a = torch.rand(3, 3)
a[a > 0.5] = 1
a[a <= 0.5] = 0

def calc_degree_matrix(a):
    return torch.diag(a.sum(dim=-1))

d = calc_degree_matrix(a)

print(a)
d

tensor([[1., 0., 0.],
        [1., 1., 1.],
        [1., 0., 0.]])


tensor([[1., 0., 0.],
        [0., 3., 0.],
        [0., 0., 1.]])

The degree matrix $D$ is used for the computation of the most important graph operator: **the graph Laplacian**!

## The grapg Laplacian

The graph Laplacian is defined as:
$$L = D - A$$

In fact, the diagonal elements of $L$ will have the degree of the node, if $A$ has no self-loops.

On the other hand, the non-diagonal elements $L_ij = -1$ , when $i \ne j $ if there is a connection. If there is **no** connection $L_ij = 0$, when $i \ne j$.

In [12]:
def calc_graph_lap(a):
    return calc_degree_matrix(a) - a

L = calc_graph_lap(a)

a = torch.Tensor(
    [
        [0., 0., 1., 1., 1.],
        [1., 0., 1., 0., 1.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 1., 0.]
        ]
    )

print(a)
L

tensor([[0., 0., 1., 1., 1.],
        [1., 0., 1., 0., 1.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 1., 0.]])


tensor([[ 3.,  0., -1., -1., -1.],
        [-1.,  3., -1.,  0., -1.],
        [-1.,  0.,  2.,  0., -1.],
        [ 0., -1.,  0.,  1.,  0.],
        [ 0.,  0., -1., -1.,  2.]])

However, in graph neural networks we use its **normalized version**. 

`Why?`

Because nodes have varying connectivity and as a result a big range of degree values on $D$.

This will create huge problems when processing with gradient-based methods.

$$L_norm = D^{-\frac {1}{2}} L D^{-\frac {1}{2}} = I - D^{-\frac {1}{2}} A D^{-\frac {1}{2}}$$

In [14]:
def calc_degree_matrix_norm(a):
 return torch.diag(torch.pow(a.sum(dim=-1),-0.5))

def create_graph_lapl_norm(a):
    size = a.shape[-1]
    D_norm = calc_degree_matrix_norm(a)
    L_norm = torch.ones(size) - (D_norm @ a @ D_norm )
    
    return L_norm

L_norm = create_graph_lapl_norm(a)

print(a)
L_norm

tensor([[0., 0., 1., 1., 1.],
        [1., 0., 1., 0., 1.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 1., 0.]])


tensor([[1.0000, 1.0000, 0.5918, 0.4226, 0.5918],
        [0.6667, 1.0000, 0.5918, 1.0000, 0.5918],
        [0.5918, 1.0000, 1.0000, 1.0000, 0.5000],
        [1.0000, 0.4226, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 0.5000, 0.2929, 1.0000]])

Now all the diagonal elements will be ones when there is at least one connection of the graph’s node independent of the node’s degree.

The node’s degree will now influence the non-diagonal elements which will be: $\frac {1}{(deg(n_i)deg(n_j))^{\frac {1}{2}}}$

In graph neural networks a slightly alternated version is often used:

$$L_{norm}^{mod} = D^{-\frac {1}{2}} (A + I) D^{-\frac {1}{2}}$$

where $I$ denotes the **identity matrix**, which adds self-connections.

From now on, we will refer to this as a `normalized graph laplacian`.

With this trick, **the input can be fed into a gradient-based algorithm without causing instabilities**.

## Laplacian eigenvalues and eigenvectors(特征值 & 特征向量)

In the most common case, **a graph that has a single zero eigenvalue is connected**, meaning that starting from any node you can visit all the other nodes with some steps.

---

# How graph convolutions layer are formed

In the simplest case we have the GCN layer:

$$Y = L_norm X W$$

$$L_{norm}^{mod} = D^{-\frac {1}{2}} (A + I) D^{-\frac {1}{2}}$$

In this case, each layer will consider only its **direct neighbors** since we use the first power of laplacian $L^1$. This is similar to a $3 \times 3$ kernel in classical image convolution, wherein we aggregate information from the direct pixel’s neighborhood.

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

def device_as(x, y):
    return x.to(y.device)

# tensor operations now support batched inputs
def calc_degree_matrix_norm(a):
    return torch.diag_embed(torch.pow(a.sum(dim=-1),-0.5))

def create_graph_lapl_norm(a):
    size = a.shape[-1]
    a +=  device_as(torch.eye(size),a)
    D_norm = calc_degree_matrix_norm(a)
    L_norm = torch.bmm( torch.bmm(D_norm, a) , D_norm )
    return L_norm

class GCN_AISUMMER(nn.Module):
    """
    A simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features, bias=bias)

    def forward(self, X, A):
        """
        A: adjecency matrix
        X: graph signal
        """
        L = create_graph_lapl_norm(A)
        x = self.linear(X)
        return torch.bmm(L, x)

Here is the total graph neural network architecture that we will use:

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class GNN(nn.Module):
    def __init__(self,
                        in_features = 7,
                        hidden_dim = 64,
                        classes = 2,
                        dropout = 0.5):
        super(GNN, self).__init__()

        self.conv1 = GCN_AISUMMER(in_features, hidden_dim)
        self.conv2 = GCN_AISUMMER(hidden_dim, hidden_dim)
        self.conv3 = GCN_AISUMMER(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, classes)
        self.dropout = dropout

    def forward(self, x,A):
        x = self.conv1(x, A)
        x = F.relu(x)
        x = self.conv2(x, A)
        x = F.relu(x)
        x = self.conv3(x, A)
        x = F.dropout(x, p=self.dropout, training=self.training)
        # aggregate node embeddings
        x = x.mean(dim=1)
        # final classification layer
        return self.fc(x)